In [1]:
import os
import import_ipynb
import pre_processing

importing Jupyter notebook from pre_processing.ipynb
GPU is installed and available
Sun Apr 16 11:00:54 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3050 T...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P0               19W /  N/A|    405MiB /  4096MiB |      8%      Default |
|                                         |                      |        

In [2]:
import keras as keras
from pre_processing import generate_training_sequences, SEQUENCE_LENGTH

In [3]:
OUTPUT_UNITS = 38
NUM_UNITS = [256,128,64]
LOSS = "sparse_categorical_crossentropy"
LEARNING_RATE = 0.001
EPOCHS = 200
BATCH_SIZE = 32
SAVE_MODEL_PATH = r"C:/Users/aswin/Downloads/Capstone - SP/Weights.h5"

In [4]:
def build_model(output_units, num_units, loss, learning_rate):
    input = keras.layers.Input(shape=(None, output_units))
    x = keras.layers.LSTM(num_units[0], return_sequences=True)(input)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.LSTM(num_units[1], return_sequences=True)(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.LSTM(num_units[2])(x)
    x = keras.layers.Dropout(0.2)(x)

    output = keras.layers.Dense(output_units, activation="softmax")(x)
    model = keras.Model(input, output)
    model.compile(loss=loss,optimizer=keras.optimizers.Adam(learning_rate=learning_rate),metrics=["accuracy"])
    model.summary()
    return model

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [6]:
def train(output_units=OUTPUT_UNITS, num_units=NUM_UNITS, loss=LOSS, learning_rate=LEARNING_RATE):
    """Train and save TF model.

    :param output_units (int): Num output units
    :param num_units (list of int): Num of units in hidden layers
    :param loss (str): Type of loss function to use
    :param learning_rate (float): Learning rate to apply
    """

    # generate the training sequences
    inputs, targets = generate_training_sequences(SEQUENCE_LENGTH)

    # build the network
    model = build_model(output_units, num_units, loss, learning_rate)

    """ train the neural network """
    # Mount Google Drive
    #drive.mount('/content/drive')

    # Create a folder to save the weights
    if not os.path.exists('C:/Users/aswin/Downloads/Capstone - SP/Weights'):
        os.makedirs('C:/Users/aswin/Downloads/Capstone - SP/Weights')

    # Define the file path for the weights
    filepath = "C:/Users/aswin/Downloads/Capstone - SP/Weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

    # Create a ModelCheckpoint callback to save weights
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )

    # Define the list of callbacks to be used during training
    callbacks_list = [checkpoint]


    # train the model
    model.fit(inputs, targets, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks_list)

    # save the model
    model.save(SAVE_MODEL_PATH)


if __name__ == "__main__":
    train()

There are 362178 sequences.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 38)]        0         
                                                                 
 lstm (LSTM)                 (None, None, 256)         302080    
                                                                 
 dropout (Dropout)           (None, None, 256)         0         
                                                                 
 lstm_1 (LSTM)               (None, None, 128)         197120    
                                                                 
 dropout_1 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_2 (Dropout)         (Non

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.